In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer, EsmForProteinFolding
from transformers.models.esm.openfold_utils.protein import to_pdb, Protein as OFProtein
from transformers.models.esm.openfold_utils.feats import atom14_to_atom37

# ✅ Set device to MPS on Mac (Metal GPU)
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"✅ Using device: {device}")

# ✅ Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("facebook/esmfold_v1")
model = EsmForProteinFolding.from_pretrained(
    "facebook/esmfold_v1", low_cpu_mem_usage=True
)
model.trunk.set_chunk_size(1)  # ✅ Smaller chunk size for low RAM
model = model.to(device)
model.eval()

# ✅ Function: Convert model output to PDB format


def convert_outputs_to_pdb(outputs):
    final_atom_positions = atom14_to_atom37(outputs["positions"][-1], outputs)
    outputs = {k: v.to("cpu").numpy() for k, v in outputs.items()}
    final_atom_positions = final_atom_positions.cpu().numpy()
    final_atom_mask = outputs["atom37_atom_exists"]

    pdbs = []
    for i in range(outputs["aatype"].shape[0]):
        aa = outputs["aatype"][i]
        pred_pos = final_atom_positions[i]
        mask = final_atom_mask[i]
        resid = outputs["residue_index"][i] + 1
        pred = OFProtein(
            aatype=aa,
            atom_positions=pred_pos,
            atom_mask=mask,
            residue_index=resid,
            b_factors=outputs["plddt"][i],
            chain_index=(
                outputs.get("chain_index", None)[i]
                if "chain_index" in outputs
                else None
            ),
        )
        pdbs.append(to_pdb(pred))
    return pdbs


# ✅ Paths
base_output_folder = "/Users/jianzhouyao/Library/Mobile Documents/com~apple~CloudDocs/Universität/ETH/Python/Digital chemistry/ESM/output"
train_output_folder = os.path.join(base_output_folder, "train")
test_output_folder = os.path.join(base_output_folder, "test")
os.makedirs(train_output_folder, exist_ok=True)
os.makedirs(test_output_folder, exist_ok=True)

train_path = "/Users/jianzhouyao/Library/Mobile Documents/com~apple~CloudDocs/Universität/ETH/Python/Digital chemistry/ESM/algpred2_train.csv"
test_path = "/Users/jianzhouyao/Library/Mobile Documents/com~apple~CloudDocs/Universität/ETH/Python/Digital chemistry/ESM/algpred2_test.csv"

# ✅ Load CSVs
df_train = pd.read_csv(train_path).dropna(subset=["sequence"])
df_test = pd.read_csv(test_path).dropna(subset=["sequence"])

# ✅ Fold train sequences
for idx, row in tqdm(
    df_train.iterrows(), total=len(df_train), desc="🔁 Processing TRAIN"
):
    pid = str(row["id"])
    sequence = row["sequence"].strip()

    if not sequence or len(sequence) < 5:
        continue

    out_file = os.path.join(train_output_folder, f"{pid}.pdb")
    if os.path.exists(out_file):
        continue

    try:
        print(f"\n🔬 Folding {pid} | Length: {len(sequence)}")
        tokenized = tokenizer(
            [sequence], return_tensors="pt", add_special_tokens=False
        )["input_ids"].to(device)

        with torch.no_grad():
            output = model(tokenized)

        pdb = convert_outputs_to_pdb(output)

        with open(out_file, "w") as f:
            f.write("".join(pdb))

        print(f"✅ Saved: {out_file}")
    except Exception as e:
        print(f"❌ Failed on {pid} | Error: {e}")

# ✅ Fold test sequences
for idx, row in tqdm(df_test.iterrows(), total=len(df_test), desc="🔁 Processing TEST"):
    pid = str(row["id"])
    sequence = row["sequence"].strip()

    if not sequence or len(sequence) < 5:
        continue

    out_file = os.path.join(test_output_folder, f"{pid}.pdb")
    if os.path.exists(out_file):
        continue

    try:
        print(f"\n🔬 Folding {pid} | Length: {len(sequence)}")
        tokenized = tokenizer(
            [sequence], return_tensors="pt", add_special_tokens=False
        )["input_ids"].to(device)

        with torch.no_grad():
            output = model(tokenized)

        pdb = convert_outputs_to_pdb(output)

        with open(out_file, "w") as f:
            f.write("".join(pdb))

        print(f"✅ Saved: {out_file}")
    except Exception as e:
        print(f"❌ Failed on {pid} | Error: {e}")

✅ Using device: mps


Some weights of EsmForProteinFolding were not initialized from the model checkpoint at facebook/esmfold_v1 and are newly initialized: ['esm.contact_head.regression.bias', 'esm.contact_head.regression.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/20150 [00:00<?, ?it/s]


🔬 Folding P_13 | Length: 562


  0%|          | 1/20150 [14:44<4950:22:24, 884.48s/it]

✅ Saved: /Users/jianzhouyao/Library/Mobile Documents/com~apple~CloudDocs/Universität/ETH/Python/Digital chemistry/ESM/output/P_13.pdb

🔬 Folding P_14 | Length: 158


  0%|          | 1/20150 [19:27<6533:12:09, 1167.28s/it]


KeyboardInterrupt: 

In [3]:
# ✅ Analyze sequence lengths
train_lengths = df_train["sequence"].dropna().apply(len)
test_lengths = df_test["sequence"].dropna().apply(len)

print("\n📊 Sequence length statistics:")
print(
    f"Train set: min = {train_lengths.min()}, max = {train_lengths.max()}, avg = {train_lengths.mean():.2f}"
)
print(
    f"Test set:  min = {test_lengths.min()}, max = {test_lengths.max()}, avg = {test_lengths.mean():.2f}"
)


📊 Sequence length statistics:
Train set: min = 50, max = 3326, avg = 354.08
Test set:  min = 50, max = 3408, avg = 338.15
